# 9.8.2 Income Redistribution Model

Agent p's reward is, $u_p(s_p - x_p) = \frac{(s_p - x_p)^{1-\alpha_p}}{1-\alpha_p}$

Wealth evolution is, $h_p(x_{p, t}, \epsilon_{p,t+1}) = \gamma_p x_{p,t} + \epsilon_{p,t+1} x_{p,t}^{\beta_p}$

Then realized wealth for agent p at time t+1 is, $s_{p, t+1} = (1 - \psi) h_p(x_{p, t}, \epsilon_{p,t+1}) + \psi h_q(x_{q, t}, \epsilon_{q,t+1})$

where $\psi$ is the pre-determined wealth share rate.

Consider two players case, each denoted by 1 and 2, who live forever. 

Then the time independent state transition function is 

$g(s_1, s_2, x_1, x_2, \epsilon_1, \epsilon_2) = ((1 - \psi) h_1(x_1, \epsilon_1)+ \psi h_2(x_2, \epsilon_2),\ (1 - \psi) h_2(x_2, \epsilon_2)+ \psi h_1(x_1, \epsilon_1))$

And using the above functions, we can set the system of Bellman equation of this problem as follows,

$\begin{cases} V_1(s_1, s_2) = \max_{0\leq x_1\leq s_1} \left\{ u_1(s_1 - x_1) + \delta E_{\epsilon}\left[ V_1(g(s_1, s_2, x_1, x_2, \epsilon_1, \epsilon_2)) \right] \right\}\\[10pt]
V_2(s_1, s_2) = \max_{0\leq x_2\leq s_2} \left\{ u_2(s_2 - x_2) + \delta E_{\epsilon}\left[ V_2(g(s_1, s_2, x_1, x_2, \epsilon_1, \epsilon_2)) \right] \right\}
\end{cases}$

I write Julia code of solving the above systems of functional equations by using collocation method.

In [19]:
# alpha, beta, gamma, psi are model's parameters
mutable struct IRG
    alpha
    beta
    gamma
    psi
end

# some functions in this model
function u(model::IRG, s::Float64, x::Float64)
    return (s - x)^(1 - model.alpha)/(1 - model.alpha)
end

function h(model::IRG, x::Float64, epsilon::Float64)
    return model.gamma * x + epsilon * (x^model.beta)
end

function s(model::IRG, s_1::Float64, s_2::Float64, x_1::Float64, x_2::Float64, epsilon_1::Float64, epsilon_2::Float64)
    return [(1 - model.psi)*h(model, x_1, epsilon_1) + model.psi*h(model, x_2, epsilon_2), 
        (1 - model.psi)*h(model, x_2, epsilon_2) + model.psi*h(model, x_2, epsilon_2)]
end

# 

s (generic function with 1 method)

In [22]:
m = IRG(0.2, 0.5, 0.9, 0.05)

IRG(0.2, 0.5, 0.9, 0.05)

In [24]:
typeof(m)

IRG

In [23]:
s(m, 2.9, 4.1, 0.3, 0.77, 0.05, 1.4)

2-element Array{Float64,1}:
 0.378592
 1.9215  

In [14]:
u(3.0,1, 5)

-0.015625